In [1]:
!pip install -q transformers datasets scikit-learn

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from torch.utils.data import Dataset

In [3]:
train_df = pd.read_csv("/content/drive/MyDrive/DATA/train_2kmZucJ.csv")
test_df = pd.read_csv("/content/drive/MyDrive/DATA/test_oJQbWVk.csv")
sample_submission = pd.read_csv("/content/drive/MyDrive/DATA/sample_submission_LnhVWA4.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
train_df.head()

Train shape: (7920, 3)
Test shape: (1953, 2)


,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [4]:
print(train_df['label'].value_counts())

label
0    5894
1    2026
Name: count, dtype: int64


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class TweetDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.encodings = tokenizer(texts.tolist(), truncation=True, padding=True, max_length=128)
        self.labels = labels.tolist() if labels is not None else None

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df["tweet"], train_df["label"], test_size=0.1, stratify=train_df["label"], random_state=42
)

train_dataset = TweetDataset(train_texts, train_labels)
val_dataset = TweetDataset(val_texts, val_labels)


In [7]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=10,
    report_to=[]  # disables W&B and other loggers
)

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    f1 = f1_score(labels, preds, average="weighted")
    return {"f1": f1}

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

import os
os.environ["WANDB_DISABLED"] = "true"


In [11]:
trainer.train()

# ✅ Inference on Test Set
test_dataset = TweetDataset(test_df["tweet"])
preds_output = trainer.predict(test_dataset)
test_preds = np.argmax(preds_output.predictions, axis=1)


Epoch,Training Loss,Validation Loss,F1
1,0.225200,0.186868,0.916829


In [13]:
submission = sample_submission.copy()
submission["label"] = test_preds
submission.to_csv("submission_1.csv", index=False)

print("✅ Submission saved to 'submission.csv'")

✅ Submission saved to 'submission.csv'


In [14]:
from google.colab import files
files.download('submission_1.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>